## 读取连接信息，构建邻接矩阵

In [1]:
import pandas as pd
import numpy as np

# 读取连接信息
connects_datas = pd.read_csv('connects.csv',)



In [2]:
L1_ports = ['D']
L2_ports = [f'X{i}' for i in range(1, 6)]
L3_ports = [f'Z{i}' for i in range(1, 74)]

In [3]:
# 三角矩阵转换为对称矩阵
def upper_to_symmetric(matrix):
    upper_triangular = np.triu(matrix)
    symmetric = upper_triangular + upper_triangular.T - np.diag(np.diag(upper_triangular))
    return symmetric


# 构建X1所在县的上三角邻接矩阵
n = 16 + 1
labels = ['X1'] + [f'Z{i}' for i in range(1, 17)]
label_to_index = {label: index for index, label in enumerate(labels)}
adj_matrix = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        adj_matrix[i,j] = -1000
for i in range(len(connects_datas)):
    if connects_datas.iloc[i, 0] in labels and connects_datas.iloc[i, 1] in labels:
        adj_matrix[label_to_index[connects_datas.iloc[i, 0]], label_to_index[connects_datas.iloc[i, 1]]] = connects_datas.iloc[i, 2]

adj_matrix = upper_to_symmetric(adj_matrix)


In [4]:
# 17个邮局
class PostOffice():
    def __init__(self, name, unload_num, load_num):
        self.name = name
        self.level = 3 if name[0] == 'Z' else 2 if name[0] == 'X' else 1
        self.unload_num = unload_num
        self.load_num = load_num
        self.load_unload_time = 5 if self.level == 3 else 10 if self.level == 2 else 0
        self.reachable_post_offices_and_distances = []
        self.get_reachable_post_offices_and_distances()
    def get_reachable_post_offices_and_distances(self):
        for i in range(len(adj_matrix)):
            if adj_matrix[i, label_to_index[self.name]] != -1000:
                self.reachable_post_offices_and_distances.append((labels[i], adj_matrix[i, label_to_index[self.name]]))


In [5]:
# 初始化17个邮局
PostOffice_in_X1_list = []
PostOffice_in_X1_list.append(PostOffice('X1', 0, 0))
PostOffice_in_X1_list.append(PostOffice('Z1', 10, 9))
PostOffice_in_X1_list.append(PostOffice('Z2', 15, 14))
PostOffice_in_X1_list.append(PostOffice('Z3', 6, 5))
PostOffice_in_X1_list.append(PostOffice('Z4', 9, 10))
PostOffice_in_X1_list.append(PostOffice('Z5', 13, 9))
PostOffice_in_X1_list.append(PostOffice('Z6', 6, 10))
PostOffice_in_X1_list.append(PostOffice('Z7', 11, 13))
PostOffice_in_X1_list.append(PostOffice('Z8', 4, 9))
PostOffice_in_X1_list.append(PostOffice('Z9', 13, 15))
PostOffice_in_X1_list.append(PostOffice('Z10', 17, 9))
PostOffice_in_X1_list.append(PostOffice('Z11', 11, 6))
PostOffice_in_X1_list.append(PostOffice('Z12', 2, 7))
PostOffice_in_X1_list.append(PostOffice('Z13', 11, 13))
PostOffice_in_X1_list.append(PostOffice('Z14', 21, 15))
PostOffice_in_X1_list.append(PostOffice('Z15', 13, 10))
PostOffice_in_X1_list.append(PostOffice('Z16', 14, 16))

In [6]:
# 邮车
class Mail_car():
    def __init__(self, capability, star_post_office):
        self.capability = capability
        self.current_load = 0
        self.current_free = capability
        self.start_post_office = star_post_office
        self.current_post_office = star_post_office

In [28]:
# 初始化16个未到达的支局
unreached_post_offices = [i for i in range(1,17)]

In [39]:
# 初始两辆车
mail_car_list = []
mail_car_list.append(Mail_car(65,0))
mail_car_list.append(Mail_car(65,0))

In [7]:
adj_matrix[8][16]

21.0

In [9]:
# # 各点之间最短路径
# def dijkstra(adj_matrix):
#     n = len(adj_matrix)
#     dist_matrix = np.zeros((n, n))
    
#     for i in range(n):
#         dist = [float('inf')] * n
#         visited = [False] * n
#         dist[i] = 0
        
#         for _ in range(n):
#             u = min((v for v in range(n) if not visited[v]), key=lambda v: dist[v])
#             visited[u] = True
            
#             for v in range(n):
#                 if adj_matrix[u][v] != -1000 and not visited[v] and dist[u] + adj_matrix[u][v] < dist[v]:
#                     dist[v] = dist[u] + adj_matrix[u][v]
        
#         dist_matrix[i] = dist

#     return dist_matrix

In [8]:
import numpy as np

def dijkstra(adj_matrix):
    n = len(adj_matrix)
    dist_matrix = np.zeros((n, n))
    path_matrix = [[[] for _ in range(n)] for _ in range(n)]
    
    for i in range(n):
        dist = [float('inf')] * n
        visited = [False] * n
        prev = [-1] * n  # 初始化前驱顶点列表
        dist[i] = 0
        
        for _ in range(n):
            u = min((v for v in range(n) if not visited[v]), key=lambda v: dist[v])
            visited[u] = True
            
            for v in range(n):
                if adj_matrix[u][v] != -1000 and not visited[v] and dist[u] + adj_matrix[u][v] < dist[v]:
                    dist[v] = dist[u] + adj_matrix[u][v]
                    prev[v] = u
        
        # 保存距离
        dist_matrix[i] = dist
        
        # 重建路径
        for end in range(n):
            path = []
            step = end
            while step != -1:
                path.append(step)
                step = prev[step]
            path_matrix[i][end] = path[::-1]  # 反转路径以显示从起始点到终点的正确顺序

    return dist_matrix, path_matrix


In [9]:
min_path_matrix, min_paths = dijkstra(adj_matrix)

In [10]:



source_node = 0
shortest_paths = min_paths[source_node]
min_path_node_list_from_X1 = shortest_paths[1:]
print("从节点 {} 到其他所有节点的最短路径节点顺序列表：".format(source_node))
for target_node, path in enumerate(shortest_paths):
    # min_path_node_list_from_X1.append(path)
    print("到节点 {} 的路径：{}".format(target_node+1, path))

从节点 0 到其他所有节点的最短路径节点顺序列表：
到节点 1 的路径：[0]
到节点 2 的路径：[0, 1]
到节点 3 的路径：[0, 3, 2]
到节点 4 的路径：[0, 3]
到节点 5 的路径：[0, 4]
到节点 6 的路径：[0, 4, 5]
到节点 7 的路径：[0, 4, 6]
到节点 8 的路径：[0, 4, 6, 7]
到节点 9 的路径：[0, 10, 8]
到节点 10 的路径：[0, 10, 9]
到节点 11 的路径：[0, 10]
到节点 12 的路径：[0, 11]
到节点 13 的路径：[0, 12]
到节点 14 的路径：[0, 13]
到节点 15 的路径：[0, 14]
到节点 16 的路径：[0, 15]
到节点 17 的路径：[0, 15, 16]


In [11]:
def cost_of_onepath(distance, loaded, load_capability):
    empty_rate = (load_capability - loaded) / load_capability
    # print(f'{empty_rate}\t{distance}')
    cost_of_onepath = 2*empty_rate*distance
    return cost_of_onepath

In [12]:
l2_car_capability = 65
# 计算环路损失
def cost_of_circle_path(node_list,if_load_label_list):
    cost = 0
    nodes_num = len(node_list)

    if node_list[0] == node_list[-1]:
        # 合并后回路
        parts_num = sum(if_load_label_list)-1
        # 计算全路径unload所需总数
        unload_spots_list = [node for index,node in enumerate(node_list) if if_load_label_list[index] == 1]
        now_loaded_num = 0
        for node in unload_spots_list:
            now_loaded_num += PostOffice_in_X1_list[node].unload_num
        now_empty_rate = (l2_car_capability - now_loaded_num) / l2_car_capability
        # 分链路依序计算cost，并相加
        now_spot = 0
        # 同空车率为一段
        for i in range(parts_num):
            distance = 0
            while(if_load_label_list[now_spot+1]==0):
                distance += adj_matrix[node_list[now_spot], node_list[now_spot+1]]
                now_spot += 1
            distance += adj_matrix[node_list[now_spot], node_list[now_spot+1]]
            now_spot += 1
            cost += 2*now_empty_rate*distance
            # print(f'{now_empty_rate}\t{distance}')
            now_loaded_num -= PostOffice_in_X1_list[node_list[now_spot]].unload_num
            now_loaded_num += PostOffice_in_X1_list[node_list[now_spot]].load_num
            now_empty_rate = (l2_car_capability - now_loaded_num) / l2_car_capability   
    else:
        # 初始单向链路 分别计算来回效益
        oneway_distance = min_path_matrix[node_list[0], node_list[-1]]
        # print(f'{node_list[0]}\t{node_list[-1]}')
        # print(oneway_distance)
        cost += cost_of_onepath(oneway_distance, PostOffice_in_X1_list[node_list[-1]].unload_num, l2_car_capability)
        cost += cost_of_onepath(oneway_distance, PostOffice_in_X1_list[node_list[-1]].load_num, l2_car_capability)
        
    return cost

# 获取两条路径合并的最好结果    
def get_save_of_combine(node_list_with_label_1,node_list_with_label_2):
    w1 = cost_of_circle_path(node_list_with_label_1[0],node_list_with_label_1[1]) 
    w2 = cost_of_circle_path(node_list_with_label_2[0],node_list_with_label_2[1])
    node_list_with_label_w3_list = get_combine_node_list_with_label_list(node_list_with_label_1,node_list_with_label_2)
    if len(node_list_with_label_w3_list) == 0:
        return 'no combine','no combine'
    combine_cost_list = [cost_of_circle_path(node_list_with_label[0],node_list_with_label[1]) for node_list_with_label in node_list_with_label_w3_list]
    best_path_index = combine_cost_list.index(min(combine_cost_list))
    best_path_cost = min(combine_cost_list)
    best_path_node_list_with_label = node_list_with_label_w3_list[best_path_index]
    save_of_combine = w1 + w2 - best_path_cost
    return save_of_combine,best_path_node_list_with_label

# 获取所有可能的路径混合 
def get_combine_node_list_with_label_list(node_list_with_label_1,node_list_with_label_2):
    node_list_1 = node_list_with_label_1[0]
    node_list_2 = node_list_with_label_2[0]
    if_load_label_list_1 = node_list_with_label_1[1]
    if_load_label_list_2 = node_list_with_label_2[1]
    # 将所有链路改写为回路
    if node_list_1[-1] != 0 :
        node_list_1 = node_list_1 + node_list_1[:-1][::-1]
        if_load_label_list_1 = if_load_label_list_1 + if_load_label_list_1[:-1][::-1]
    if node_list_2[-1] != 0 :
        node_list_2 = node_list_2 + node_list_2[:-1][::-1]
        if_load_label_list_2 = if_load_label_list_2 + if_load_label_list_2[:-1][::-1]
    # 定位两条回路的四个紧接着县局的需装卸的支局index
    # print(if_load_label_list_1)
    # print(if_load_label_list_2)
    start_spot_1 = if_load_label_list_1[1:-1].index(1) + 1
    end_spot_1 = len(node_list_1) - if_load_label_list_1[::-1][1:-1].index(1) - 2
    start_spot_2 = if_load_label_list_2[1:-1].index(1) + 1
    end_spot_2 = len(node_list_2) - if_load_label_list_2[::-1][1:-1].index(1) - 2
    # print(start_spot_1,end_spot_1,start_spot_2,end_spot_2)
# [1,2,3,4,5]
    # 全视为环路
    possible_path =  [
        [node_list_1[:end_spot_1+1] + node_list_2[start_spot_2:],if_load_label_list_1[:end_spot_1+1] + if_load_label_list_2[start_spot_2:]],
        [node_list_1[:end_spot_1+1] + node_list_2[:end_spot_2+1][::-1],if_load_label_list_1[:end_spot_1+1] + if_load_label_list_2[:end_spot_2+1][::-1]],
        [node_list_2[:end_spot_2+1] + node_list_1[start_spot_1:],if_load_label_list_2[:end_spot_2+1] + if_load_label_list_1[start_spot_1:]],
        [node_list_2[:end_spot_2+1] + node_list_1[:end_spot_1+1][::-1],if_load_label_list_2[:end_spot_2+1] + if_load_label_list_1[:end_spot_1+1][::-1]]
    ]
    # print(result)
    # print(node_list_1[:end_spot_1+1])
    # print(node_list_2[start_spot_2:])
    # 四种情况讨论
    # if node_list_1[-1] != 0 and node_list_2[-1] != 0:
    #     # 两种合并链路，互为反向
    #     result =  [[node_list_1 + node_list_2[::-1] , if_load_label_list_1 + if_load_label_list_2[::-1]],[node_list_2 + node_list_1[::-1] , if_load_label_list_2 + if_load_label_list_1[::-1]]]
    # # 第二种，前者为初始链回路，后者为环路
    # elif node_list_1[-1] != 0 and node_list_2[-1] == 0:
    #     # 四种合并链路
    #     result =  [
    #         [node_list_1 + node_list_2[1:],if_load_label_list_1 + if_load_label_list_2[1:]],
    #         [node_list_1 + node_list_2[:-1][::-1],if_load_label_list_1 + if_load_label_list_2[:-1][::-1]],
    #         [node_list_2[:-1] + node_list_1[::-1],if_load_label_list_2[:-1] + if_load_label_list_1],
    #         [node_list_2[1:][::-1] + node_list_1[::-1],if_load_label_list_2[1:][::-1] + if_load_label_list_1[::-1]]
    #     ]
    # # 第三种，前者为环路，后者为初始链回路
    # elif node_list_1[-1] == 0 and node_list_2[-1] != 0:
    #     # 四种合并链路
    #     result =  [
    #         [node_list_2 + node_list_1[1:],if_load_label_list_2 + if_load_label_list_1[1:]],
    #         [node_list_2 + node_list_1[:-1][::-1],if_load_label_list_2 + if_load_label_list_1[:-1][::-1]],
    #         [node_list_1[:-1] + node_list_2[::-1],if_load_label_list_1[:-1] + if_load_label_list_2],
    #         [node_list_1[1:][::-1] + node_list_2[::-1],if_load_label_list_1[1:][::-1] + if_load_label_list_2[::-1]]
    #     ]
    # # 第四种，都是环路
    # elif node_list_1[-1] == 0 and node_list_2[-1] == 0:
    #     # 四种合并链路
    #     result =  [
    #         [node_list_1[:-1]] + node_list_2[1:],if_load_label_list_1[:-1] + if_load_label_list_2[1:],
    #         [node_list_1[:-1]] + node_list_2[:-1][::-1],if_load_label_list_1[:-1] + if_load_label_list_2[:-1][::-1],
    #         [node_list_2[:-1]] + node_list_1[1:],if_load_label_list_2[:-1] + if_load_label_list_1[1:],
    #         [node_list_2[:-1]] + node_list_1[:-1][::-1],if_load_label_list_2[:-1] + if_load_label_list_1[:-1][::-1]
    #     ]
    
    # 有重复装卸的路径删除
    # 先取出所有路径的装卸点
    main_spots_list = [[x for x, mask in zip(node_list_with_label[0], node_list_with_label[1]) if mask == 1] for node_list_with_label in possible_path]
    
    # 合并后的路径必然0为起点和终点,删除有除0外的重复装卸点的路径
    result = [x for x, main_spots in zip(possible_path, main_spots_list) if len(set(main_spots)) == len(main_spots) -1]
    # 然后补充仅仅是增加装卸点的路径
    for node_list_with_label in possible_path:
        if node_list_with_label[0] == node_list_1 or node_list_with_label[0] == node_list_2:
            result.append(node_list_with_label)
    result = del_invalid_path(result)
    return result

def check_if_overload(node_list, if_load_label_list):
    now_load = 0
    # 出发前装载要送达沿途装卸支局的货物
    for i in range(len(node_list)):
        if if_load_label_list[i] == 1:
            now_load += PostOffice_in_X1_list[node_list[i]].unload_num
    for i in range(len(node_list)):
        if now_load > l2_car_capability:
            return True
        if if_load_label_list[i] == 1:
            now_load -= PostOffice_in_X1_list[node_list[i]].unload_num
            now_load += PostOffice_in_X1_list[node_list[i]].load_num
    return False

def check_if_timeout(node_list, if_load_label_list):
    # print(node_list)
    path_lists = []
    for i in range(len(node_list)-1):
        tmp_distance = adj_matrix[node_list[i], node_list[i+1]]
        if tmp_distance < 0:
            return True
        path_lists.append(tmp_distance)
    sum_distance = sum(path_lists)
    main_l3_posts_num = sum(if_load_label_list) - 2
    # 总分钟数
    spend_time = (sum_distance * 2) + main_l3_posts_num*5 +20
    if 360 >= spend_time:
        return False
    else:
        return True


def del_invalid_path(node_list_with_label_list):
    result = []
    for node_list_with_label in node_list_with_label_list:
        node_list, if_load_label_list = node_list_with_label[0], node_list_with_label[1]
        timeout_label = check_if_timeout(node_list, if_load_label_list)
        overload_label = check_if_overload(node_list, if_load_label_list)
        if not timeout_label and not overload_label:
            result.append(node_list_with_label)
    return result



In [13]:
print(adj_matrix[4][5])
print(adj_matrix[5][6])
print(adj_matrix[5][7])
print(adj_matrix[6][7])
print(adj_matrix[4][6])


13.0
9.0
21.0
13.0
20.0


In [ ]:
adj_matrix[4][5]

In [14]:
min_path_node_list_from_X1

[[0, 1],
 [0, 3, 2],
 [0, 3],
 [0, 4],
 [0, 4, 5],
 [0, 4, 6],
 [0, 4, 6, 7],
 [0, 10, 8],
 [0, 10, 9],
 [0, 10],
 [0, 11],
 [0, 12],
 [0, 13],
 [0, 14],
 [0, 15],
 [0, 15, 16]]

In [15]:
import copy
now_path_list = copy.deepcopy(min_path_node_list_from_X1)

In [153]:
min_path_node_list_from_X1

[[0, 1],
 [0, 3, 2],
 [0, 3],
 [0, 4],
 [0, 4, 5],
 [0, 4, 6],
 [0, 4, 6, 7],
 [0, 10, 8],
 [0, 10, 9],
 [0, 10],
 [0, 11],
 [0, 12],
 [0, 13],
 [0, 14],
 [0, 15],
 [0, 15, 16]]

In [154]:
# now_path_list_with_label_list = [[i,[1]+[0]*(len(i)-2)+[1]]  for i in now_path_list]


In [155]:
now_path_list_with_label_list

[[[0, 1], [1, 1]],
 [[0, 3, 2], [1, 0, 1]],
 [[0, 3], [1, 1]],
 [[0, 4], [1, 1]],
 [[0, 4, 5], [1, 0, 1]],
 [[0, 4, 6], [1, 0, 1]],
 [[0, 4, 6, 7], [1, 0, 0, 1]],
 [[0, 10, 8], [1, 0, 1]],
 [[0, 10, 9], [1, 0, 1]],
 [[0, 10], [1, 1]],
 [[0, 11], [1, 1]],
 [[0, 12], [1, 1]],
 [[0, 13], [1, 1]],
 [[0, 14], [1, 1]],
 [[0, 15], [1, 1]],
 [[0, 15, 16], [1, 0, 1]]]

In [101]:
import random

def choose_one_combine(now_path_list_with_label_list,rate = None, now_min_cost = None):
    saved_cost_list = []
    new_path_list = []
    combine_path_index = []
    combine_flag = False
    for i in range(len(now_path_list_with_label_list)):
        for j in range(i+1,len(now_path_list_with_label_list)):
            path_1 = now_path_list_with_label_list[i]
            path_2 = now_path_list_with_label_list[j]
            # 剪枝，待合并的两条路都比当前now_min_cost大,就跳过
            # if now_min_cost:
            #     if cost_of_circle_path(path_1[0],path_1[1]) > now_min_cost and cost_of_circle_path(path_2[0],path_2[1]) > now_min_cost :
            #         continue
            save_of_combine,best_path_node_list_with_label = get_save_of_combine(path_1,path_2)
            if save_of_combine == 'no combine':
                continue
            else:
                saved_cost_list.append(save_of_combine)
                new_path_list.append(best_path_node_list_with_label)
                combine_path_index.append((i,j))
                combine_flag = True
    if not combine_flag:
        return now_path_list_with_label_list, combine_flag
    sorted_indices = sorted(range(len(saved_cost_list)), key=lambda i: saved_cost_list[i],reverse=True)
    max_saved_combine_index = sorted_indices[0]
    if rate:
        if len(saved_cost_list) != 1:
            available_choice_num = len(saved_cost_list)
            if available_choice_num >= len(rate):
                random_value = random.randint(0,sum(rate))
                selected_index = 0
                for i in range(len(rate)):
                    random_value -= rate[i]
                    if random_value <= 0:
                        break
                    selected_index += 1
            if available_choice_num < len(rate):
                random_value = random.randint(0,sum(rate[:available_choice_num]))
                selected_index = 0
                for i in range(available_choice_num):
                    random_value -= rate[i]
                    if random_value <= 0:
                        break
                    selected_index += 1
            max_saved_combine_index= sorted_indices[selected_index]
    # 确定合并的两条路径下标
    combined_path_indexes = combine_path_index[max_saved_combine_index]
    # print(f'1 combine:\n {now_path_list_with_label_list[combined_path_indexes[1]]} \n {now_path_list_with_label_list[combined_path_indexes[0]]} \n {new_path_list[max_saved_combine_index]}')
    del now_path_list_with_label_list[combined_path_indexes[1]]
    del now_path_list_with_label_list[combined_path_indexes[0]]
    # 为合并的路径检查是否可优化（2-opt）
    optimized_path = opt_optimization(new_path_list[max_saved_combine_index])

    now_path_list_with_label_list.append(optimized_path)
    
    return now_path_list_with_label_list, combine_flag

def opt_optimization(path_list_with_label):
    mid_node_num = len(path_list_with_label[0])
    min_cost = cost_of_circle_path(path_list_with_label[0],path_list_with_label[1])
    min_path_node_list = path_list_with_label[0]
    min_path_load_label_list = path_list_with_label[1]
    for i in range(1,mid_node_num-2):
        for j in range(i+1,mid_node_num-1):
            new_path_node_list = copy.deepcopy(path_list_with_label[0])
            new_path_node_list = new_path_node_list[0:i] + new_path_node_list[i:j+1][::-1] + new_path_node_list[j+1:]
            new_path_load_label_list = copy.deepcopy(path_list_with_label[1])
            new_path_load_label_list = new_path_load_label_list[0:i] + new_path_load_label_list[i:j+1][::-1] + new_path_load_label_list[j+1:]
            if check_if_overload(new_path_node_list,new_path_load_label_list) or check_if_timeout(new_path_node_list,new_path_load_label_list):
                continue
            
            new_cost = cost_of_circle_path(new_path_node_list,new_path_load_label_list)
            if  new_cost < min_cost:
                # print('2-opt activate')
                # print(f'from path {min_path_node_list} to {new_path_node_list}')
                # print(f'from load {min_path_load_label_list} to {new_path_load_label_list}')
                # print(f'from cost {min_cost} to {new_cost}')

                min_cost = new_cost
                min_path_node_list = new_path_node_list
                min_path_load_label_list = new_path_load_label_list
                
    return [min_path_node_list,min_path_load_label_list]




            

In [102]:
try_once(1)

(155.2923076923077,
 [[[0, 4], [1, 1]],
  [[0, 4, 6, 7, 8, 9, 16, 10, 0], [1, 0, 1, 1, 1, 1, 1, 0, 1]],
  [[0, 3, 2, 5, 15, 14, 12, 0], [1, 0, 1, 1, 1, 1, 1, 1]],
  [[0, 13, 1, 3, 10, 11, 0], [1, 1, 1, 1, 1, 1, 1]]],
 1)

In [182]:
now_path_list_with_label_list = [[i,[1]+[0]*(len(i)-2)+[1]]  for i in now_path_list]

while True:
    new_path_list_with_label_list, updated_flag = choose_one_combine(now_path_list_with_label_list)
    if not updated_flag:
        break
    now_path_list_with_label_list = new_path_list_with_label_list


1 combine:
 [[0, 10, 8], [1, 0, 1]] 
 [[0, 4, 5, 7], [1, 0, 0, 1]] 
 [[0, 10, 8, 7, 5, 4, 0], [1, 0, 1, 1, 0, 0, 1]]
1 combine:
 [[0, 10, 8, 7, 5, 4, 0], [1, 0, 1, 1, 0, 0, 1]] 
 [[0, 15, 16], [1, 0, 1]] 
 [[0, 15, 16, 8, 7, 5, 4, 0], [1, 0, 1, 1, 1, 0, 0, 1]]
1 combine:
 [[0, 15, 16, 8, 7, 5, 4, 0], [1, 0, 1, 1, 1, 0, 0, 1]] 
 [[0, 10, 9], [1, 0, 1]] 
 [[0, 10, 9, 16, 8, 7, 5, 4, 0], [1, 0, 1, 1, 1, 1, 0, 0, 1]]
1 combine:
 [[0, 10, 9, 16, 8, 7, 5, 4, 0], [1, 0, 1, 1, 1, 1, 0, 0, 1]] 
 [[0, 4, 6], [1, 0, 1]] 
 [[0, 4, 6, 7, 8, 16, 9, 10, 0], [1, 0, 1, 1, 1, 1, 1, 0, 1]]
1 combine:
 [[0, 4, 5], [1, 0, 1]] 
 [[0, 3, 2], [1, 0, 1]] 
 [[0, 3, 2, 5, 4, 0], [1, 0, 1, 1, 0, 1]]
1 combine:
 [[0, 3, 2, 5, 4, 0], [1, 0, 1, 1, 0, 1]] 
 [[0, 1], [1, 1]] 
 [[0, 1, 2, 5, 4, 0], [1, 1, 1, 1, 0, 1]]
1 combine:
 [[0, 1, 2, 5, 4, 0], [1, 1, 1, 1, 0, 1]] 
 [[0, 14], [1, 1]] 
 [[0, 1, 2, 5, 14, 0], [1, 1, 1, 1, 1, 1]]
1 combine:
 [[0, 15], [1, 1]] 
 [[0, 11], [1, 1]] 
 [[0, 15, 11, 0], [1, 1, 1, 1]]
1 co

In [17]:
import threading

# 创建锁对象
# lock = threading.Lock()



def try_once(args):
    rate = [16,8,4,2,1]
    index = args
    now_path_list_with_label_list = [[i,[1]+[0]*(len(i)-2)+[1]]  for i in now_path_list]
    while True:
        new_path_list_with_label_list, updated_flag = choose_one_combine(now_path_list_with_label_list,rate=rate)
        if not updated_flag:
            break
        now_path_list_with_label_list = new_path_list_with_label_list
    now_cost = 0
    for i in now_path_list_with_label_list:
        now_cost += cost_of_circle_path(i[0],i[1])
    return now_cost, now_path_list_with_label_list,index
    

In [173]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from IPython.display import clear_output
test_num = 10000
count = 0
# cost_and_path_list = [None]*test_num
global min_cost,min_path_list
min_cost = 1000
cost_list = [None]*test_num
path_list = [None]*test_num
min_path_list = None
with ThreadPoolExecutor(max_workers=32) as executor:
    futures = [executor.submit(try_once, i) for i in range(test_num)]
    for future in as_completed(futures):
        count += 1
        now_cost, now_path_list_with_label_list,i = future.result()
        cost_list[i] = now_cost
        if now_cost < min_cost:
            min_cost = now_cost
            min_path_list = now_path_list_with_label_list
        # Calculate the percentage of completion
        percent_complete = (count / test_num) * 100
        # Print the real-time percentage progress
        if count % 10 == 0:
            clear_output(wait=True)
            print(f"Progress: {percent_complete:.2f}% Complete ({count}/{test_num})")


Progress: 100.00% Complete (10000/10000)


In [45]:
from IPython.display import clear_output
import concurrent.futures
import threading

# 创建锁对象
lock = threading.Lock()
test_num = 1000
count = 0
# cost_and_path_list = [None]*test_num
global min_cost,min_path_list
min_cost = 1000
min_path_list = None


# 定义线程池，max_workers参数设置线程池中最多同时运行的线程数
with concurrent.futures.ThreadPoolExecutor(max_workers=32) as executor:
    # 提交任务给线程池，并获取结果
    count = 0
    results = executor.map(try_once, range(1, test_num))
    # 处理每一个结果
    for result in results:
        now_cost, now_path_list_with_label_list,i = result
        lock.acquire()
        if now_cost < min_cost:
            min_cost = now_cost
            min_path_list = now_path_list_with_label_list
        lock.release()
        count += 1
        percent_complete = (count / test_num) * 100
        # # Print the real-time percentage progress
        if count % 100 == 0:
            clear_output(wait=True)
            print(f"Progress: {percent_complete:.2f}% Complete ({count}/{test_num})")

Progress: 90.00% Complete (900/1000)


In [46]:
min_cost

58.338461538461544

In [40]:
min_path_list

[[[0, 12, 11, 15, 16, 8, 9, 10, 0], [1, 1, 1, 1, 1, 1, 1, 0, 1]],
 [[0, 4, 6, 7, 5, 10, 0], [1, 1, 1, 1, 1, 1, 1]],
 [[0, 13, 1, 2, 3, 14, 0], [1, 1, 1, 1, 1, 1, 1]]]

In [103]:
# 非多线程实现
from tqdm import tqdm
min_cost = 1000
min_path = None

for i in tqdm(range(1000)):
    now_path_list_with_label_list = [[i,[1]+[0]*(len(i)-2)+[1]]  for i in now_path_list]
    while True:
        new_path_list_with_label_list, updated_flag = choose_one_combine(now_path_list_with_label_list,rate=[16,8,4,2,1], now_min_cost=min_cost)
        if not updated_flag:
            break
        now_path_list_with_label_list = new_path_list_with_label_list
    now_cost = 0
    for i in now_path_list_with_label_list:
        now_cost += cost_of_circle_path(i[0],i[1])
        if now_cost > min_cost:
            break
    if now_cost < min_cost:
        min_cost = now_cost
        min_path = now_path_list_with_label_list
        print(f'min_cost updated to {min_cost}')
   

  1%|          | 8/1000 [00:00<00:32, 30.37it/s]

min_cost updated to 244.5846153846154
min_cost updated to 139.35384615384618
min_cost updated to 107.35384615384616
min_cost updated to 79.44615384615386


  2%|▏         | 20/1000 [00:00<00:31, 30.70it/s]

min_cost updated to 74.12307692307692


  6%|▋         | 65/1000 [00:02<00:29, 31.56it/s]

...

 19%|█▉        | 193/1000 [00:05<00:22, 35.94it/s]

min_cost updated to 60.400000000000006


 40%|███▉      | 397/1000 [00:11<00:16, 36.37it/s]

min_cost updated to 57.753846153846155


100%|██████████| 1000/1000 [00:28<00:00, 35.44it/s]


In [104]:
for i in tqdm(range(10000)):
    now_path_list_with_label_list = [[i,[1]+[0]*(len(i)-2)+[1]]  for i in now_path_list]
    while True:
        new_path_list_with_label_list, updated_flag = choose_one_combine(now_path_list_with_label_list,rate=[16,8,4,2], now_min_cost=min_cost)
        if not updated_flag:
            break
        now_path_list_with_label_list = new_path_list_with_label_list
    now_cost = 0
    if len(now_path_list_with_label_list)>3:
        continue
    for i in now_path_list_with_label_list:
        now_cost += cost_of_circle_path(i[0],i[1])
        if now_cost > min_cost:
            break
    if now_cost < min_cost:
        min_cost = now_cost
        min_path = now_path_list_with_label_list
        print(f'min_cost updated to {min_cost}')

100%|██████████| 10000/10000 [05:31<00:00, 30.16it/s]


In [105]:
print(min_cost)
print(min_path) 

57.753846153846155
[[[0, 12, 11, 15, 16, 9, 8, 10, 0], [1, 1, 1, 1, 1, 1, 1, 0, 1]], [[0, 4, 6, 7, 5, 10, 0], [1, 1, 1, 1, 1, 1, 1]], [[0, 13, 1, 2, 3, 14, 0], [1, 1, 1, 1, 1, 1, 1]]]


In [163]:
print(adj_matrix[0][4])
print(adj_matrix[4][5])
print(adj_matrix[5][6])
print(adj_matrix[4][6])
print(adj_matrix[9][0])

11.0
13.0
9.0
20.0
-1000.0


In [180]:
now_path_list_with_label_list

[[[0, 4, 6, 7, 8, 16, 9, 10, 0], [1, 0, 1, 1, 1, 1, 1, 0, 1]],
 [[0, 3, 1, 2, 5, 14, 0], [1, 1, 1, 1, 1, 1, 1]],
 [[0, 4, 10, 15, 11, 12, 13, 0], [1, 1, 1, 1, 1, 1, 1, 1]]]

In [167]:
all_cost = 0
for i in now_path_list_with_label_list:
    all_cost += cost_of_circle_path(i[0],i[1])

In [168]:
all_cost

90.8923076923077

In [176]:
print(cost_of_circle_path([0,4,5,2,3,1,13,0],[1,1,1,1,1,1,1,1]))
print(cost_of_circle_path([0,4,5,6,7,8,16,10,0],[1,0,0,1,1,1,1,1,1]))
print(cost_of_circle_path([0,12,11,15,9,14,0],[1,1,1,1,1,1,1]))

19.323076923076922
27.107692307692307
20.83076923076923


In [177]:
check_if_timeout([0,4,5,6,7,8,16,10,0],[1,0,0,1,1,1,1,1,1])

False

In [179]:
check_if_overload([0,4,6,7,8,16,10,0],[1,0,1,1,1,1,1,1])

False

In [166]:
19.323076923076922+27.107692307692307+20.83076923076923

67.26153846153846

In [155]:
check_if_overload([0, 4, 10, 15, 11, 12, 13, 0], [1, 1, 1, 1, 1, 1, 1, 1])

False

In [120]:
cost_of_circle_path([0, 4, 9, 8, 10, 11, 12, 0], [1, 1, 1, 1, 1, 1, 1, 1])

0.13846153846153847	11.0
0.12307692307692308	28.0
0.09230769230769231	11.0
0.015384615384615385	20.0
0.13846153846153847	18.0
0.2153846153846154	23.0
0.13846153846153847	21.0


33.292307692307695

In [117]:
cost_of_circle_path([0, 1, 0], [1, 1 ,1])

0.8461538461538461	27.0
0.8615384615384616	27.0


92.21538461538462

In [101]:
adj_matrix[0][1]

27.0